In [1]:
#export
"""
This module is for optics simulation. This is exposed automatically with::

   from k1lib.imports import *
   kop.Rays.parallel(...) # exposed

A comprehensive introduction is available here: https://mlexps.com/optics/1-kop-intro/"""
import k1lib, time, math, html, copy; from k1lib import cli, p5; import numpy as np; from collections import deque
__all__ = ["Drawable", "RandomPoints", "Drawables",
           "RaysFocus", "Rays", "Wavelengths", "sellmeier", "gps",
           "Surface", "LineSurface", "ArcSurface", "polySolver", "PolySurface", "RaysPath",
           "OpticElement", "Polygon", "Lens", "OpticSystem"]

In [2]:
#export
colormap_rgb = np.array([[6, 1, 31],[12, 0, 40],[14, 0, 51], [16, 1, 60], [17, 1, 76], [23, 0, 90], [26, 1, 105], [28, 0, 119], [28, 0, 136], [34, 0, 151],[36, 1, 165], [37, 0, 176], [37, 1, 187], [36, 0, 194], [37, 0, 202], [34, 0, 209], [31, 0, 217], [28, 1, 220], [25, 0, 224], [18, 1, 227], [16, 0, 229], [14, 0, 233], [10, 0, 237], [9, 0, 237], [7, 0, 240], [3, 0, 242], [0, 0, 244], [0, 0, 244], [2, 5, 244], [1, 8, 244], [0, 13, 242], [0, 18, 242], [2, 22, 239], [0, 28, 236], [0, 33, 236], [0, 37, 232], [0, 44, 229], [2, 49, 227], [0, 55, 220], [0, 60, 218], [0, 66, 214], [1, 73, 209], [0, 77, 205], [0, 84, 200], [0, 91, 194], [0, 96, 193], [0, 101, 189], [0, 106, 182], [0, 111, 177], [1, 118, 172], [0, 120, 165], [0, 122, 159], [0, 128, 153], [1, 131, 147], [1, 132, 140], [1, 135, 134], [0, 140, 131], [0, 145, 126], [0, 148, 124], [0, 152, 122], [0, 158, 118], [1, 162, 118], [1, 168, 116], [0, 172, 114], [0, 178, 113],[0, 182, 112], [0, 186, 111], [1, 188, 109], [2, 191, 107], [0, 194, 107], [1, 195, 108], [0, 198, 101], [1, 200, 99], [0, 204, 96], [1, 209, 97], [2, 211, 94], [1, 217, 90], [0, 220, 88], [0, 225, 81], [1, 228, 77], [1, 231, 71], [1, 232, 68], [0, 230, 60], [0, 230, 52], [0, 230, 43], [0, 230, 33], [0, 228, 21], [0, 228, 11], [2, 229, 0], [16, 229, 0], [28, 229, 0], [40, 230, 0], [56, 232, 0], [72, 232, 2], [84, 230, 1], [98, 231, 0],[111, 230, 0],[124, 230, 0], [137, 230, 1], [151, 228, 0], [162, 227, 0], [173, 229, 0], [186, 227, 0], [198, 224, 1], [211, 226, 0], [221, 221, 0], [227, 216, 0], [230, 210, 1], [237, 201, 1], [240, 193, 1],[242, 184, 0], [245, 173, 0], [248, 165, 1], [250, 155, 0], [251, 145, 0], [252, 136, 1], [254, 126, 1], [255, 115, 0], [255, 104, 3], [254, 95, 1], [255, 83, 1], [255, 72, 2], [255, 61, 0], [253, 49, 0], [255, 39, 2], [253, 28, 0], [255, 17, 4], [255, 8, 1], [254, 2, 1], [254, 0, 10], [255, 0, 14], [255, 0, 18], [251, 0, 24], [250, 0, 30], [250, 0, 30], [248, 0, 35], [246, 0, 41], [246, 0, 41], [242, 0, 40], [242, 0, 40], [240, 0, 45], [237, 0, 46], [233, 0, 45], [230, 1, 44], [226, 0, 42], [222, 0, 41], [218, 0, 39], [214, 0, 38], [206, 0, 36], [200, 1, 34], [195, 0, 32], [189, 0, 30], [185, 0, 31], [177, 0, 28], [169, 0, 26], [162, 0, 24], [152, 0, 23],[144, 1, 21], [136, 1, 18], [128, 1, 20], [121, 0, 19], [111, 0, 16], [104, 0, 14], [96, 0, 12], [88, 1, 10], [83, 0, 12], [73, 0, 9], [67, 0, 9], [62, 1, 9], [57, 0, 7], [51, 0, 7], [46, 0, 5], [42, 0, 4], [39, 0, 5], [33, 1, 4], [30, 0, 4],[25, 0, 3], [25, 0, 3], [22, 0, 2],[21, 0, 1], [16, 0, 0], [15, 1, 1], [14, 0, 0], [12, 0, 0], [9, 0, 1], [9, 0, 1], [8, 0, 0]])
colormap_nm = np.linspace(400,700,len(colormap_rgb)) # courtesy of https://stackoverflow.com/questions/71977306/how-to-convert-rgb-to-wavelength-in-python
def wavelength_to_rgb(wavlgth): return np.asarray(colormap_rgb[min(range(len(colormap_nm)), key=lambda i: abs(colormap_nm[i]-wavlgth))])
class Wavelengths: # vectorized version of wavelength_to_rgb
    def __init__(self, wavs:"List[int]"):
        """Container for wavelengths in nm. Pretty much only used to convert to rgb in vectorized way"""
        self.wavs = np.array(wavs | cli.deref())
    def _toRgb(self):
        wavs = self.wavs; single = len(wavs.shape) == 0
        if single: wavs = wavs[None]
        ans = colormap_rgb[abs(colormap_nm[None]-wavs[:,None]) | cli.toArgmin().all()]
        return ans[0] if single else ans
    def __repr__(self): return f"<Wavelengths #={len(self.wavs)} min={round(self.wavs.min(), 2)}nm max={round(self.wavs.max(), 2)}nm std={round(self.wavs.std(), 5)}nm>"

In [3]:
cli.init.patchNumpy()
cli.init.patchDict()
assert (wavelength_to_rgb(300) == wavelength_to_rgb(200)).all()
assert (Wavelengths([470, 500]) | cli.toRgb() == np.array([[  0,  84, 200], [  1, 168, 116]])).all()

In [4]:
#export
pi = 3.141592653589793; inf = float("inf"); settings = k1lib.Settings(); k1lib.settings.add("kop", settings, "from k1lib.kop module, for optics-related tools");
settings.add("display", k1lib.Settings(), "display settings").add("consts", k1lib.Settings(), "magic constants throughout the sim. By default works pretty well, but you can tweak these if you need unrealistic setups, like super big focal length, etc")
settings.display.add("drawable", k1lib.Settings()
                     .add("axes", True, "whether to add x and y axes to the sketch")
                     .add("maxWh", 800, "when drawing a sketch, it will be rescaled so that the maximum of width and height of the final image is this number. Increase to make the sketch bigger")
                     .add("grid", True, "whether to add grid lines to the sketch")
                     .add("gridColor", (255, 255, 255)), "generic draw settings")
class Drawable:
    """Base class to do common drawing routines"""
    def bounds(self):
        """Should return (x1, y1, x2, y2). Base classes should always implement this method"""
        return NotImplemented
    def _draw(self, config): return NotImplemented
    def _preprocess(self):
        xmin, ymin, xmax, ymax = bounds = self.bounds(); dx = xmax-xmin; dy = ymax-ymin; s = settings.display.drawable.maxWh/max(dx, dy)
        c = {"depth": 0, "bounds": bounds}; config = {**self.config, **c} if isinstance(self, Drawables) else c
        p5.newSketch(dx, dy, pad=20, scale=s, xoff=xmin, yoff=ymin); p5.background(222); p5.textSize(12/s)
        xm = xmin-20/s; ym = ymin-20/s # xmin and ymin without padding
        try: xticks = k1lib.ticks(xmin, xmax); bsx = max(int(len(xticks)/(dx*s/30)), 1) # this is for when the width or height is too small to draw, then this will only draw a tick for every bsx ticks available
        except: xticks = [xmin]; bsx = 1 # if xmin == xmax
        try: yticks = k1lib.ticks(ymin, ymax); bsy = max(int(len(yticks)/(dy*s/30)), 1)
        except: yticks = [ymin]; bsy = 1
        if settings.display.drawable.grid:
            with p5.context():
                p5.stroke(*settings.display.drawable.gridColor)
                for x in xticks | cli.batched(bsx, True) | cli.item().all(): p5.line(x, ymin-20/s, x, ymax+20/s);
                for y in yticks | cli.batched(bsy, True) | cli.item().all(): p5.line(xmin-20/s, y, xmax+20/s, y);
        if settings.display.drawable.axes:
            for x in xticks | cli.batched(bsx, True) | cli.item().all(): p5.line(x, ymin, x, ym); p5.text(f"{x}", x+5/s, ym+5/s)
            for y in yticks | cli.batched(bsy, True) | cli.item().all(): p5.line(xmin, y, xm, y); p5.text(f"{y}", xm+5/s, y+5/s)
        self._draw(config)
    def _repr_extra(self): return ""
    def _repr_html_(self): self._preprocess(); r = html.escape(f"{self}"); return f"{r}{self._repr_extra()}<br>{p5.svg()}"
    def img(self) -> "PIL.Image.Image":
        """Returns PIL image"""
        self._preprocess(); return p5.img()
    def svg(self) -> str:
        """Returns svg string"""
        self._preprocess(); return p5.svg()
class RandomPoints(Drawable): # displays random points
    def __init__(self, data):
        """Container to place random points to be graphed either alone or inside a :class:`Drawables`"""
        self.data = data | cli.deref() | cli.toNdArray()
    def _draw(self, config): [p5.ellipse(x,y,0.5) for x,y in self.data]; return p5.svg()
    def bounds(self): return min(self.data[0]), min(self.data[1]), max(self.data[0]), max(self.data[1])
    def __copy__(self): return RandomPoints(np.copy(self.data))
class Drawables(Drawable): # a container to just draw everything out
    def __init__(self, drawables:"List[Drawable]", config=None):
        """A container with multiple :class:`Drawable` s, to draw out everything possible within a single coordinate frame"""
        self.drawables = drawables; self.config = config or {}
    def __copy__(self): return Drawables([copy.copy(d) for d in self.drawables])
    def bounds(self): return self.drawables | cli.op().bounds().all() | cli.T() | cli.toMin() + cli.toMin() + cli.toMax() + cli.toMax() | cli.deref()
    def _draw(self, config): [d._draw({**config, **self.config}) for d in self.drawables]; return p5.svg()
    def __repr__(self): return f"<Drawables #n={len(self.drawables)}>"
settings.add("colorD", {"rainbow": [400, 650], "red": [620, 750], "orange": [590, 620], "yellow": [570, 590], "green": [495, 570], "blue": [450, 495], "purple": [400, 450]}, "color wavelength ranges to be used in constructing Rays")
def color_interpreter(col, N):
    if isinstance(col, str):
        if col not in settings.colorD: raise Exception(f"Does not understand the color '{col}'. Current options are: {', '.join(settings.colorD.keys())}. You can set your own custom colors using `settings.kop.colorD`")
        return np.linspace(*settings.colorD[col], N)
    if isinstance(col, (int, float, np.number)): return np.ones(N)*col
    if isinstance(col, (list, tuple)) and len(col) == 2: return np.linspace(*col, N)
    raise Exception(f"Does not understand the color/wavelength '{col}'. It can be a string like 'red', a specific wavelength in nm (450), or a list/tuple with 2 floats specifying start and end wavelengths")
settings.display.add("rays", k1lib.Settings().add("showOrigin", True, "whether to add a small red dot to the beginning of the mean (x,y) of a Rays or not").add("infLength", 100, "length in mm to display Rays if their length is infinite"), "display settings of kop.Rays")
class RaysFocus(Drawable):
    def __init__(self, rays:"Rays"):
        r = rays; x1 = r.data[:,0][None]; x2 = r.data[:,0][:,None]; y1 = r.data[:,1][None]; y2 = r.data[:,1][:,None]; t1 = r.data[:,2][None]; t2 = r.data[:,2][:,None]
        with k1lib.ignoreWarnings(): det = 1/(-np.cos(t1)*np.sin(t2)+np.sin(t1)*np.cos(t2)); self.rays = rays
        invalids = det==inf; det[invalids] = 0; a = det*-np.sin(t2); b = det*np.cos(t2); c = det*-np.sin(t1); d = det*np.cos(t1)
        alpha = a*(x2-x1) + b*(y2-y1); self.xs = (x1+alpha*np.cos(t1))[~invalids]; self.ys = (y1+alpha*np.sin(t1))[~invalids]; self.xy = np.stack([self.xs, self.ys])
        xmin, ymin, xmax, ymax = self.bounds(); self.xmin = xmin; self.xmax = xmax; self.ymin = ymin; self.ymax = ymax; self.xmedian = np.median(self.xs); self.ymedian = np.median(self.ys)
        self.dx = xmax - xmin; self.dy = ymax - ymin; self.x = (xmin+xmax)/2; self.y = (ymin+ymax)/2; self.xstd = self.xy[0].std(); self.ystd = self.xy[1].std()
    def bounds(self): return self.xy[0].min(), self.xy[1].min(), self.xy[0].max(), self.xy[1].max()
    def _draw(self, config):
        bounds = config["bounds"]; pointSize = math.sqrt((bounds[2]-bounds[0])*(bounds[3]-bounds[1]))/100
        for x, y in self.xy.T: p5.ellipse(x, y, pointSize)
    def __repr__(self):
        xdigits = max(int(-math.log10(self.dx))+2, 0); ydigits = max(int(-math.log10(self.dy))+2, 0)
        return f"<RaysFocus N={len(self.xy[0])} xrange=({round(self.xmin, xdigits)}, {round(self.xmedian, xdigits)}, {round(self.xmax, xdigits)}) xstd={round(self.xstd, 6)} yrange=({round(self.ymin, ydigits)}, {round(self.ymedian, ydigits)}, {round(self.ymax, ydigits)}) ystd={round(self.ystd, 6)}>"
class Rays(Drawable):
    def __init__(self, data, prevRays:"Rays"=None, ogSurface:"Surface"=None):
        """Represents a bunch of rays, stored in a numpy array so that all
operations are fast

`data` format: array of shape (N, 6)
- 0) x coordinate of starting point
- 1) y coordinate of starting point
- 2) angle counter clockwise from positive x axis
- 3) length (inf for forever)
- 4) wavelength in nm
- 5) #transforms
- 6) power in Watts, defaulted to 1W
- 7) intersected?: used for outgoing rays exiting out of a surface. If surface._cast(prevRay) shows that it does intersect, then add that to the outgoing rays

The #transforms is a little complicated:
- Original ray is 0, then each time a glass/mirror surface does something interesting,
the outgoing ray is incremented by 1. If it doesn't touch the optic element, then
it keeps the same number as before

:param prevRays: a reference to the previous Rays object, in order to limit the length of all previous rays!
:param ogSurface: the surface that generates this Rays"""
        self.data = data; self.prevRays = prevRays; self.ogSurface = ogSurface
    def __copy__(self): return Rays(np.copy(self.data), None if self.prevRays is None else copy.copy(self.prevRays), self.ogSurface)
    @staticmethod
    def _genElse(array, color=700, power=1): N = len(array); return Rays(array | cli.insertColumn([inf]*N, False) | cli.insertColumn(color_interpreter(color, N), False) | cli.insertColumn([0]*N, False) | cli.insertColumn([power]*N, False) | cli.insertColumn([0]*N, False))
    @staticmethod
    def parallel(x=0, y=0, theta=0, N=10, height=10, color=700, power=1):
        """Creates parallel rays starting from a particular point with a particular angle

:param theta: angle from positive x axis counterclockwise
:param N: how many rays to create total?
:param height: the height of the parallel rays if it were to have no angle"""
        a = theta + pi/2; b = [x+math.cos(a)*height/2, y+math.sin(a)*height/2]
        a = theta - pi/2; c = [x+math.cos(a)*height/2, y+math.sin(a)*height/2]
        return Rays._genElse([np.linspace(b[0], c[0], N), np.linspace(b[1], c[1], N)] | cli.toNdArray() | cli.T() | cli.insertColumn([theta]*N, False), color=color, power=power)
    @staticmethod
    def parallelToBounds(x=0, y=0, bounds=None, N=10, color=700, power=1, coverage=0.9, angleOffset=0):
        """Creates parallel rays starting from a particular point to the center of some bounds.
The bounds should have the format (xmin, ymin, xmax, ymax). :class:`Surface`s, :class:`OpticElement` s
all have the .bounds() method, so you can use them

See also: :meth:`parallel`

:param coverage: number from 0 to 1. 1 meaning the parallel lines should cover the entire (rotated)
    height of the bounds, 0.5 means it covers only half of the height, and so on. Think of this as %height"""
        if bounds is None: raise Exception(".bounds parameter is required")
        x1, y1, x2, y2 = bounds; xs = np.array([x1, x2, x2, x1]); ys = np.array([y1, y1, y2, y2]); angles = np.arctan2(ys-y, xs-x)%(2*pi)
        amin = angles.argmin(); x1 = xs[amin]; y1 = ys[amin]; amax = angles.argmax(); x2 = xs[amax]; y2 = ys[amax]
        dx = x2-x1; mx = (x1+x2)/2; dy = y2-y1; my = (y1+y2)/2; angle = math.atan2(my-y, mx-x)
        h = math.sin(angle+pi-math.atan2(y2-y1, x2-x1))*math.sqrt(dx*dx+dy*dy)
        return Rays.parallel(x, y, angle+angleOffset, N, h*coverage, color, power)
    @staticmethod
    def pointToBounds(x=0, y=0, bounds=None, N=10, color=700, power=1, coverage=0.9, angleOffset=0):
        """Creates rays starting from a particular point to the bounds.

See also: :meth:`parallelToBounds`

:param coverage: instead of %height like :meth:`parallelToBounds`, this is the anglular coverage,
    1 for covering the entire object.
:param angleOffset: offset to the angle from the starting point to the center of the bounds"""
        if bounds is None: raise Exception(".bounds parameter is required")
        x1, y1, x2, y2 = bounds; xs = np.array([x1, x2, x2, x1]); ys = np.array([y1, y1, y2, y2]); angles = np.arctan2(ys-y, xs-x)#%(2*pi)
        amin = angles.argmin(); x1 = xs[amin]; y1 = ys[amin]; amax = angles.argmax(); x2 = xs[amax]; y2 = ys[amax]
        startAngle = math.atan2(y1-y, x1-x) % (2*pi); endAngle = math.atan2(y2-y, x2-x) % (2*pi)
        if endAngle < startAngle: endAngle += 2*pi
        angleDiff = (endAngle - startAngle)*(1-coverage)
        return Rays._genElse(np.array([[x]*N, [y]*N, np.linspace(startAngle+angleDiff/2+angleOffset, endAngle-angleDiff/2+angleOffset, N)]).T, color, power)
    def bounds(self): # returns bounds for plotting by p5
        with k1lib.ignoreWarnings(): l = np.copy(self.data[:,3]); l[l==inf] = settings.display.rays.infLength
        data = self.data; a = data[:,0] + np.cos(data[:,2])*l; b = data[:,1] + np.sin(data[:,2])*l
        xmin = min(data[:,0].min(), a.min()); xmax = max(data[:,0].max(), a.max())
        ymin = min(data[:,1].min(), b.min()); ymax = max(data[:,1].max(), b.max())
        return xmin, ymin, xmax, ymax
    def hitbox(self, spread=2, maxWH=400, mode="simple", intersectOnly=True) -> "PIL.Image.Image":
        """Visualizes the Ray's starting point, with colors and whatnot.
Example::

    r = Rays.parallel(theta=pi/8)
    r.hitbox()

:param spread: will color this much nearby pixels
:param maxWH: size of the returned image. Either width or height will be around this big, whichever is bigger
:param mode: several plotting modes, each with its own performance and accuracy characteristics
:param intersectOnly: if True (default), will only plot the hitbox of the rays that actually hits
    the Surface that generates the ray and not some previous surface
"""
        if mode == "simple":
            if self.data[:,7].sum()<0.5: raise Exception("No ray intersected the Surface that generates this Rays")
            data = self.data[self.data[:,7]>0.5][:,[0, 1, 4, 6]]; xy = data[:,:2]
            xmin = min(xy[:,0]); xmax = max(xy[:,0]); xmid = (xmin+xmax)/2; dx = xmax-xmin
            ymin = min(xy[:,1]); ymax = max(xy[:,1]); ymid = (ymin+ymax)/2; dy = ymax-ymin
            N = len(data); dist = np.sum((xy[None] - xy[:,None])**2, 2)**0.5; dist[range(N), range(N)] = inf; dm = dist.min()
            if maxWH is None: maxWH = min(max(dx/dm, dy/dm), 500); maxWH # number of pixels of the longest length. This is calculated automatically if user does not give a particular size
            pixelSize = max(dx, dy)/maxWH # how much real distance is between 2 adjacent pixels? Ideally exactly the same as dm
            pixelH = int(max(dy/pixelSize*1.1, 20)); pixelW = int(max(dx/pixelSize*1.1, 20))
            im = np.ones((pixelH, pixelW, 3))*200; rgb = Wavelengths(data[:,2]) | cli.toRgb()
            ys = (-(xy[:,1]-ymid)/pixelSize+pixelH/2).astype(int); xs = ((xy[:,0]-xmid)/pixelSize+pixelW/2).astype(int)
            for i in np.arange(1+spread*2)-spread:
                for j in np.arange(1+spread*2)-spread: im[ys+i, xs+j] = rgb
            return im | cli.toImg()
        else: raise Exception(f"Currently doesn't support mode '{mode}'")
    def focus(self) -> "RaysFocus": return RaysFocus(self)
    def __add__(self, rays):
        if not isinstance(rays, Rays): raise Exception(f"Can only add Rays to another Rays! The other value has type {type(rays)}")
        return Rays(np.concatenate([self.data, rays.data]))
    def __repr__(self): return f"<Rays N={len(self.data)} avgTheta={self.data | cli.cut(2) | cli.toMean() | cli.op()/pi*180 | cli.aS(round, 2)}deg intersectedSurface?={self.data[:,7].sum()>0.5} ogSurface={self.ogSurface}>"
    def _draw(self, config={}, ignored=None):
        with k1lib.ignoreWarnings(): data = self.data; l = np.copy(self.data[:,3]); l[l==inf] = settings.display.rays.infLength; bounds = config["bounds"]
        a = data[:,0] + np.cos(data[:,2])*l; b = data[:,1] + np.sin(data[:,2])*l; xmin, ymin, xmax, ymax = self.bounds()
        with p5.context():
            for params in [data[:,0], data[:,1], a, b, data[:,4], ignored if ignored is not None else [False]*len(data)] | cli.T() | ~cli.filt("x", 5):
                p5.stroke(*wavelength_to_rgb(params[4])); p5.line(*params[:4])
            if settings.display.rays.showOrigin: p5.fill(255, 0, 0); p5.noStroke(); p5.ellipse(*data[:, :2] | cli.T() | cli.toMean().all(), math.sqrt((bounds[2]-bounds[0])*(bounds[3]-bounds[1]))/100)
        r = html.escape(f"{self}"); return p5.svg()

In [5]:
r = Rays.parallel(x=10, theta=pi/16, color=490); r

<Rays N=10 avgTheta=11.25deg intersectedSurface?=False ogSurface=None>

In [6]:
#export
gps = {"BK7": (1.03961212, 0.231792344, 1.01046945, 0.00600069867, 0.0200179144, 103.560653), # borosilicate crown glass
               "sapphire": (1.4313493, 0.65054713, 5.3414021, 0.0052799261, 0.0142382647, 325.017834),
               "fusedSilica": (0.6961663, 0.4079426, 0.8974794, 0.00467914826, 0.0135120631, 97.9340025),
               "MgF": (0.48755108, 0.39875031, 2.3120353, 0.001882178, 0.008951888, 566.13559),
               "air": (0,0,0,0,0,0)}
settings.add("gps", gps, "All builtin glass parameters of the system. All have 6 floats, for B1,B2,B3,C1,C2,C3 parameters used in the sellmeier equation: https://en.wikipedia.org/wiki/Sellmeier_equation")
_invGlassParams = [None, 0]
def invGlassParams():
    global gps, _invGlassParams
    if time.time() - _invGlassParams[1] > 1: gps = {**settings.gps, **gps}; settings.gps = gps; _invGlassParams = [{v:k for k, v in gps.items()}, time.time()]
    return _invGlassParams[0]
def sellmeier(wavelengths:"np.ndarray", glassParam):
    """Runs the sellmeier equation to get the index of refraction for multiple wavelengths
for a particular glass material"""
    if isinstance(glassParam, (int, float, np.number)): return np.array([glassParam]*len(wavelengths))
    else:
        la2 = (wavelengths/1000)**2; b1, b2, b3, c1, c2, c3 = glassParam
        return (1 + b1*la2/(la2-c1) + b2*la2/(la2-c2) + b3*la2/(la2-c3))**0.5
sellmeier(np.linspace(400, 700, 10), gps["BK7"])

array([1.53084854, 1.52694023, 1.52387508, 1.52141448, 1.51939901,
       1.51771854, 1.51629483, 1.51507095, 1.51400466, 1.513064  ])

In [7]:
#export
_surface_autoInc = k1lib.AutoIncrement(prefix="_surface_")
def snell(rtheta, ln, n1, n2): # this takes into account whether we've encountered total internal reflection or not
    a = n1*np.sin(rtheta-ln)/n2; b = (ln + np.arcsin(a)).clearNan(); inRange = (-1 <= a) * (a <= 1)
    f = -2*np.cos(rtheta-ln); c = np.arctan2(np.sin(rtheta)+f*np.sin(ln), np.cos(rtheta)+f*np.cos(ln))
    return inRange * b + (1-inRange) * c
settings.display.add("surface", k1lib.Settings().add("showIndex", True, "whether to show the index of the Surface in an OpticSystem or not"), "display settings for kop.Surface class")
settings.consts.add("inchForward", 1e-6, "after new Rays have been built, inch forward the origin of the new Rays by a this tiny amount so that it 'clears' the last Surface")
class Surface(Drawable):
    def __init__(self, gp1=None, gp2=None, mode="glass", capture=False, angleStd=0, opticElement=None):
        """Represents a geometry, like line, arc, or parabola. Rays can interact with a
surface, resulting in new Rays. An optic element can have multiple of these surfaces and
react differently to incoming rays.

The convension is that gp1 is the environment's glassParam, while gp2 is the Surface's
glassParam. A surface needs both in order to calculate the deflection angles.

These modes are available:

- glass: transparent (or semi-transparent) surface, bends incoming rays
- mirror: perfectly reflective (or allow some percentage of light to pass through)
- sensor: absorbes all incoming rays, can retrieve the image that's absorbed
- opaque: reflects incoming rays in a random direction outward

:param gp1: glass params 1, usually this is of the environment (air)
:param gp2: glass param 2, usually this is of the object (borosilicate glass, sapphire, water)
:param mode: explained above
:param capture: if True, captures all rays that comes in through .cast() function. Captures
    will stay the same across copy.copy() instances to save perf! The captures will be
    available at ``.captures``
:param angleStd: if specified, when casting outgoing rays, will add this much standard deviation
    in the rays' angle, useful to quickly simulate a bumpy piece of glass
:param opticElement: the OpticElement that this Surface is a part of"""
        self.idx = _surface_autoInc()
        self.gp1 = (self.gp1 or gp1) if hasattr(self, "gp1") else gp1
        self.gp2 = (self.gp2 or gp2) if hasattr(self, "gp2") else gp2
        self.mode = (self.mode or mode) if hasattr(self, "mode") else mode
        self.capture = capture; self.captures = []; self.angleStd = angleStd; self.opticElement = opticElement
    def __copy__(self): res = super().__new__(type(self)); res.idx = self.idx; res.gp1 = self.gp1; res.gp2 = self.gp2; res.mode = self.mode; res.capture = self.capture; res.captures = self.captures; res.angleStd = self.angleStd; res.opticElement = self.opticElement; return res
    def _cast(self, rays:"Rays"):
        """Expected to be implemented by all subclasses.
Given N rays, should return these arrays:
- (N,) bool array: whether the rays intersect the surface or not
- (N,) float array: length of the previous ray if intersect. Else can be any number, but please no whacky values like inf or nan
- (N,) float array: angle of vector normal to the surface. Say it points up. Then top area is considered gp1, bottom area is gp2. If don't intersect then can be any number

This should be enough to totally and completely describe the surface properties, and the refraction
code only needs to be written once"""
        return NotImplemented
    def _cast2(self, rays):
        intersected, beta, ln = self._cast(rays)
        beta.clearNan(); ln.clearNan(); beta[beta==inf] = 0; ln[ln==inf] = 0
        return intersected, beta, ln
    def castS(self, rays) -> "Rays":
        """This will truncate the lengths of the incoming Rays object, and return
a new rays, as well as the lengths of the old rays, for easy comparison. If the
rays don't interact with the lens at all, then just copy the old rays over, and the
length is the same."""
        if self.capture: self.captures.append([copy.copy(rays)])
        if self.mode == "glass":
            with k1lib.ignoreWarnings():
                if self.gp1 is None: raise Exception(f"gp1 not specified in {self}!")
                if self.gp2 is None: raise Exception(f"gp2 not specified in {self}!")
                r = rays; intersected, beta, ln = self._cast2(rays); rtheta = r.data[:,2]; N = len(r.data)
                n1 = sellmeier(r.data[:,4], self.gp1); n2 = sellmeier(r.data[:,4], self.gp2) # index of refractions
                lns = np.stack([ln, pi+ln]).T; ln_version = abs((rtheta[:,None]-lns+pi)%(2*pi)-pi) | cli.toArgmin().all()
                d1to2 = ln_version # whether the ray is going from gp1 (glass param 1, top of line) to gp2. Completely coincidentally, this is exactly equal to ln_version (line normal version, either as-is, or flipped 180)
                chosenLn = lns[range(N), ln_version] # actual normal that it's going to use
                t1 = snell(rtheta, chosenLn, n1, n2); t2 = snell(rtheta, chosenLn, n2, n1)
                rthetap = d1to2*t1 + (1-d1to2)*t2; intersections = [r.data[:,0]+np.cos(r.data[:,2])*(beta+1e-8), r.data[:,1]+np.sin(r.data[:,2])*(beta+1e-8)] | cli.toNdArray() | cli.T()
        elif self.mode == "mirror":
            with k1lib.ignoreWarnings():
                r = rays; intersected, beta, ln = self._cast2(rays); rtheta = r.data[:,2]; N = len(r.data); lns = np.stack([ln, pi+ln]).T
                angleDiff = (rtheta[:,None]-lns+pi)%(2*pi)-pi; ln_version = abs(angleDiff) | cli.toArgmin().all()
                f = 2*np.cos(angleDiff[range(N), ln_version]) # factor to multiply the normal vector by
                chosenLn = lns[range(N), 1-ln_version] # actual normal that it's going to use
                rthetap = np.arctan2(np.sin(rtheta)+f*np.sin(chosenLn), np.cos(rtheta)+f*np.cos(chosenLn))
                intersections = [r.data[:,0]+np.cos(r.data[:,2])*(beta-1e-8), r.data[:,1]+np.sin(r.data[:,2])*(beta-1e-8)] | cli.toNdArray() | cli.T()
        else: raise Exception(f"Doesn't support mode '{self.mode}' yet")
        with k1lib.ignoreWarnings(): # make sure intersected, intersections, rthetap variables are defined upon reaching this part
            if abs(self.angleStd) > 1e-11: rthetap = rthetap + np.random.randn(rthetap.shape)*self.angleStd # adding variations
            intersections[:,0] += np.cos(rthetap)*settings.consts.inchForward; intersections[:,1] += np.sin(rthetap)*settings.consts.inchForward # inch forward a little to clear the last surface
            outr = np.concatenate([intersections | cli.insertColumn(rthetap, False), r.data[:,3:]], 1) # has not corrected for whether it has intersected the line
            outr[~intersected] = np.copy(r.data[~intersected]); outr[:,3] = float("inf"); outr[intersected, 5] = r.data[intersected, 5] + 1; outr[~intersected, 5] = r.data[~intersected, 5]; outr[:,7] = intersected; outr = Rays(outr, r, self)
            # tries to limit the length of previous rays
            r.data[:,3] = beta * intersected + (r.data[:,3] * (1 - intersected)).clearNan(); newR = r; r = r.prevRays
            while r is not None: # for the rays that does not have its depth incremented, set the prevLength to this lengths
                idxs = np.abs(newR.data[:,5] - r.data[:,5]) < 1e-9; r.data[idxs,3] = newR.data[idxs,3]; newR = r; r = r.prevRays
            if self.capture: self.captures[-1].append(copy.copy(outr))
        return outr
    def cast(self, rays, **kwargs) -> "RaysPath": # convenience function, not core mechanism
        return OpticSystem().add(self).cast(rays, **kwargs)
    def _draw(self, config):
        if settings.display.surface.showIndex and "surfaceIdx2Id" in config:
            with p5.context(): xmin, ymin, xmax, ymax = self.bounds(); p5.fill(100); p5.noStroke(); p5.text(config["surfaceIdx2Id"][self.idx], (xmin+xmax)/2, (ymin+ymax)/2)
    def _repr(self): # helper sections at the end of every surface
        gp1 = f" gp1=None" if self.gp1 is None else (f" gp1='{invGlassParams()[self.gp1]}'" if self.gp1 in invGlassParams() else "")
        gp2 = f" gp2=None" if self.gp2 is None else (f" gp2='{invGlassParams()[self.gp2]}'" if self.gp2 in invGlassParams() else "")
        oe = "" if self.opticElement is None else f' opticElement={self.opticElement}'
        return f"{gp1}{gp2}{' capture=True' if self.capture else ''}{oe}"

In [8]:
#export
def ray_intersection(line:"list[float]", rays:Rays):
    with k1lib.ignoreWarnings():
        lx1, ly1, lx2, ly2 = line; dlx = lx2 - lx1; dly = ly2 - ly1
        if abs(dlx) < 1e-11: dlx += 1e-10 # surprisingly, it still works for such tiny numbers!
        R = dly/dlx; rx = rays.data[:,0]; ry = rays.data[:,1]; rtheta = rays.data[:,2]
        beta = (ry-ly1+R*(lx1-rx))/(np.cos(rtheta)*R-np.sin(rtheta)) # vector length of ray
        alpha = (rx+beta*np.cos(rtheta)-lx1)/dlx; return alpha, beta # vector length in line surface
class LineSurface(Surface):
    def __init__(self, x1, y1, x2, y2, **kwargs):
        """Surface defined using a straight line.

If x1,y1 is on the left, x2,y2 on the right, then gp1 (glass param) is on top, gp2 is
at the bottom."""
        if not hasattr(self, "_superAlreadyInit"): super().__init__(**kwargs)
        self.x1 = x1; self.y1 = y1; self.x2 = x2; self.y2 = y2
    def __copy__(self): res = super().__copy__(); res._superAlreadyInit = True; res.__init__(self.x1, self.y1, self.x2, self.y2); return res
    def _cast(self, rays):
        l = self; alpha, beta = ray_intersection([l.x1, l.y1, l.x2, l.y2], rays)
        ln = (math.atan2(l.y2 - l.y1, l.x2 - l.x1) + pi/2) % (2*pi) # line normal angle
        intersected = (0 <= alpha) * (alpha <= 1) * (beta > 0); return intersected, beta, np.ones(len(rays.data))*ln
    def bounds(self): return min(self.x1, self.x2), min(self.y1, self.y2), max(self.x1, self.x2), max(self.y1, self.y2)
    def _draw(self, config): super()._draw(config); p5.line(self.x1, self.y1, self.x2, self.y2); return p5.svg()
    def __repr__(self): return f"<Line(Surface) ({round(self.x1, 2)}, {round(self.y1, 2)}, {round(self.x2, 2)}, {round(self.y2, 2)}){self._repr()}>"
class ArcSurface(Surface):
    def __init__(self, x, y, r, startAngle, endAngle, **kwargs):
        """Surface defined using a circular arc"""
        if not hasattr(self, "_superAlreadyInit"): super().__init__(**kwargs)
        self.x = x; self.y = y; self.r = r
        startAngle = startAngle%(2*pi); endAngle = endAngle-2*pi # ensuring startAngle is always positive, and endAngle is always greater than startAngle. This defines the angle sweeped counterclockwise from startAngle to endAngle
        while endAngle < startAngle: endAngle += 2*pi
        self.startAngle = startAngle; self.endAngle = endAngle
    @staticmethod
    def from2Points(x1, y1, x2, y2, r, **kwargs) -> "ArcSurface":
        """Creates an :class:`ArcSurface` from 2 points and a radius.
Example::

    kop.ArcSurface.from2Points(1, 2, 3, 4, 10)
"""
        if r < 0: x1,y1,x2,y2 = x2,y2,x1,y1; r = -r
        mx = (x1+x2)/2; dx = x2-x1; my = (y1+y2)/2; dy = y2-y1; dist = math.sqrt(dx*dx+dy*dy); a = math.atan2(dy, dx)-pi/2; h2 = r*r-dist*dist/4
        if h2 < 0: raise Exception(f"Arc radius {r} too small. No arc will pass through both of the specified points")
        h = math.sqrt(h2); cx = mx + h*math.cos(a); cy = my + h*math.sin(a); return ArcSurface(cx, cy, r, math.atan2(y2-cy, x2-cx), math.atan2(y1-cy, x1-cx), **kwargs)
    def __copy__(self): res = super().__copy__(); res._superAlreadyInit = True; res.__init__(self.x, self.y, self.r, self.startAngle, self.endAngle); return res
    def _cast(self, rays):
        rx, ry, rtheta = rays.data[:,:3].T; c = np.cos(rtheta); s = np.sin(rtheta); ax = self.x; ay = self.y; ar = self.r; N = len(rx)
        A = rx-ax; B = ry-ay; a = 1; b = 2*(A*c+B*s); c = A*A+B*B-ar*ar; delta = b*b-4*a*c; intersected = delta > 0 # quadratic formula
        with k1lib.ignoreWarnings(): delta = np.sqrt(delta).clearNan(); v1 = (-b-delta)/(2*a); v2 = (-b+delta)/(2*a)
        v1[v1<0]=inf; v1[~intersected]=inf; v2[v2<0]=inf; v2[~intersected]=inf
        vs = np.stack([v1,v2]).T; tx = rx[:,None]+vs*np.cos(rtheta[:,None]); ty = ry[:,None]+vs*np.sin(rtheta[:,None])
        angles = np.arctan2(ty-ay, tx-ax); angles[angles<self.startAngle] += 2*pi; angles[angles<self.startAngle] += 2*pi; oldAngles = angles
        selected_version = (self.startAngle < angles) * (angles < self.endAngle) * (-inf < vs) * (vs < inf) | cli.toArgmax().all()
        angles = angles[range(N), selected_version]; intersected[~((self.startAngle < angles) * (angles < self.endAngle))] = False
        beta = vs[range(N), selected_version]; intersected[beta==inf] = False; beta[beta==inf] = 0; return intersected, beta, angles%(2*pi)
    def bounds(self): # step through n=50 points from startAngle to endAngle, then calc the bounds
        angles = np.linspace(self.startAngle, self.endAngle); a = self.x+self.r*np.cos(angles); b = self.y+self.r*np.sin(angles); return a.min(), b.min(), a.max(), b.max()
    def _draw(self, config):
        super()._draw(config);
        with p5.context(): p5.noFill(); p5.arc(self.x,self.y,self.r,self.startAngle,self.endAngle); return p5.svg()
    def __repr__(self): return f"<Arc(Surface) x={round(self.x, 2)} y={round(self.y, 2)} r={round(self.r, 2)} startAngle(deg)={round(self.startAngle/pi*180, 2)} endAngle(deg)={round(self.endAngle/pi*180, 2)}{self._repr()}>"

In [9]:
#export
def buildF(coeff): # compiles function of the passed in coefficients
    s = " "
    for i, c in enumerate(coeff):
        if c != 0: s += f" + {c}*x**{i}"
    s += " "; s = s.replace("*x**0 ", " ").replace("**1 ", " ").replace("**2 ", "*x ").strip(" +")
    return eval(f"lambda x: {s if s else '0'}")
def buildDF(coeff): # compiles the derivative function of the passed in coefficients
    s = " "
    for i, c in enumerate(coeff[1:]):
        if c != 0: s += f" + {c*(i+1)}*x**{i}"
    s += " "; s; s = s.replace("*x**0 ", " ").replace("**1 ", " ").replace("**2 ", "*x ").strip(" +")
    return eval(f"lambda x: {s if s else '0'}")
def polySolver(polys):
    """Expected to take in a (N,F) array of N different polynomials with order F-1 and return 2 arrays:

- intersected (N,) bool array: whether there're any roots
- beta (N,) float array: the root of each polynomial that's >0 and is the minimum out of all roots

Right now, only supports vectorized solvers with order 2 polynomials. Any higher and it
falls back to using un-vectorized np.roots(), or other methods like it. This is separated out
so that it can be swapped out for a better implementation

Potential candidates for >2 order polynomials:
- scipy: fsolve, root, newton, brentq
"""
    order = polys.shape[1]-1; N = polys.shape[0]
    if order == 2:
        c,b,a = polys.T
        intersected = np.array([True]*N); delta = b*b-4*a*c; intersected[delta < 0] = False
        delta[delta < 0] = 0; delta = np.sqrt(delta); x1 = (-b-delta)/(2*a); x2 = (-b+delta)/(2*a); xs = np.stack([x1, x2]).T; xs[xs<0] = inf
        beta_version = xs | cli.toArgmin().all(); beta = xs[range(N), beta_version]; intersected[beta==inf] = False; 
        beta = beta.clearNan(); beta[beta==inf] = 0; beta = (abs(a)<1e-8)*(-c/b).clearNan() + (abs(a)>=1e-8)*beta; intersected[beta<0] = False
        return intersected, beta
    else: raise Exception("Currently only support polynomials of order 2, aka a parabola. Why aren't higher dimensions supported? Cause I'm lazy. You can copy this function (polySolver), create your own robust version that supports higher dimensions and insert it into PolySurface.solver")
class PolySurface(Surface):
    def __init__(self, x=0, y=0, angle=0, scale=1, xmin=-10, xmax=10, coeff=(0,0,0.1), solver=polySolver, **kwargs):
        """Surface defined using a polynomial.

Let's say there's a parabola that you'd like to input into the simulation, say ``x^2/10``. There
are several dials and knobs that you can change to position your Surface right where you want it.

There are 2 frame of reference I'll be using. First is world frame, where all your elements sits in,
and second is poly frame, where your polynomial coefficients will define (x, y) in.

In the poly frame, the surface is defined as all x points in (xmin, xmax), together with all y points
calculated straight from your coefficients. So if ``coeff = [1, 2, 3]``, then the equation will be
``py = f(px) = 3px^2 + 2px + 1``. Now pairs of (px, py) points in poly frame is obtained.

Then (px, py) will go through several transformations to get to the world frame:
- Rotated by ``angle`` radians counterclockwise
- Scaled by ``scale``
- Translated by ``(x, y)``

Then to calculate reflections and whatnot, internally, all rays will be translated from world to poly
frame, then the intersection points are solved and then results will be translated from poly back
to world frame.

:param x: translation applied to surface
:param angle: angle to rotate the surface by
:param scale: scaling factor to scale the surface by
:param xmin: to define the domain of the given polynomial in poly frame
:param coeff: polynomial coefficients"""
        if not hasattr(self, "_superAlreadyInit"): super().__init__(**kwargs)
        if len(coeff) < 2: raise Exception(f"This only works with polynomial of order 2 or higher. Use the simpler :class:`LineSurface` for order 1!")
        self.x = x; self.y = y; self.angle = angle; self.scale = scale; self.xmin = xmin; self.xmax = xmax; self.coeff = tuple(coeff); self.solver = solver
        self.f = buildF(coeff); self.df = buildDF(coeff)
    @staticmethod
    def parabola(x, y, r=5, width=16, angle=0, **kwargs): return PolySurface(x, y, angle, 1, -width/2, width/2, (0, 0, 1/(2*r)), **kwargs)
    @staticmethod
    def parabolaFrom2Points(x1, y1, x2, y2, r=5, **kwargs):
        dx = x2-x1; mx = (x1+x2)/2; dy = y2-y1; my = (y1+y2)/2
        dist = math.sqrt(dx*dx+dy*dy); a = -1/(2*r); angle = math.atan2(dy, dx); height = a*(dist/2)**2
        x = mx + height*math.cos(angle-pi/2); y = my + height*math.sin(angle-pi/2)
        return PolySurface(x, y, angle, 1, -dist/2, dist/2, (0, 0, a), **kwargs)
    def __copy__(self): res = super().__copy__(); res._superAlreadyInit = True; res.__init__(self.x, self.y, self.angle, self.scale, self.xmin, self.xmax, self.coeff, self.solver); return res
    def world2poly(self, wxs, wys): c = math.cos(-self.angle); s = math.sin(-self.angle); xs = (wxs-self.x)/self.scale; ys = (wys-self.y)/self.scale; return xs*c-ys*s, xs*s+ys*c
    def poly2world(self, pxs, pys): c = math.cos(self.angle); s = math.sin(self.angle); return self.x+self.scale*(pxs*c-pys*s), self.y+self.scale*(pxs*s+pys*c)
    def _cast(self, rays):
        rx, ry = self.world2poly(rays.data[:,0], rays.data[:,1]); rtheta = rays.data[:,2]-self.angle; N = len(rx); coeff = self.coeff
        # this clusterfuck piece of code is trying to solve f(rx+beta*cos(rtheta))-(ry+beta*sin(rtheta))=0. I don't want to use numerical
        # methods as those are not vectorized, so I have to rewrite the whole equation in terms of beta and then do the vectorized
        # analytical quadratic formula later on. This code builds up that polynomial array where the variable is beta
        a = np.cos(rtheta); b = rx; n = order = len(coeff)-1; polys = np.zeros((N, order+1)) # first column is x^0, last row is x^n
        for ex in range(n+1): # trying to expand c0*(ax+b)^0 + c1*(ax+b)^1 + ... + cn*(ax+b)^n
            for k in range(ex+1): polys[:,ex-k] += coeff[ex]*math.comb(ex, k)*a**(n-k)*b**k
        polys[:,0] -= ry; polys[:,1] -= np.sin(rtheta) # then adding some correction term. Once polynomials are formed, then beta can be solved, and normal angles can be found
        intersected, beta = self.solver(polys); tx = rx+beta*np.cos(rtheta); intersected[((tx < self.xmin) | (tx > self.xmax))] = False
        tx = np.maximum(np.minimum(tx, self.xmax), self.xmin); angles = np.arctan(self.df(tx))+pi/2+self.angle; return intersected, beta*self.scale, angles%(2*pi)
    def bounds(self): pxs = np.linspace(self.xmin, self.xmax); xs, ys = self.poly2world(pxs, self.f(pxs)); return min(xs), min(ys), max(xs), max(ys)
    def _draw(self, config):
        super()._draw(config); pxs = np.linspace(self.xmin, self.xmax); xs, ys = self.poly2world(pxs, self.f(pxs));
        [xs, ys] | cli.T() | cli.window(2) | cli.joinSt().all() | ~cli.apply(p5.line) | cli.ignore(); return p5.svg()
    def __repr__(self): return f"<Poly(Surface) coeff={self.coeff} x={round(self.x, 2)} y={round(self.y, 2)} angle(deg)={round(self.angle/pi*180, 2)} scale={round(self.scale, 2)} xmin={round(self.xmin, 2)} xmax={round(self.xmax, 2)}{self._repr()}>"

In [10]:
#export
class RaysPath(Drawable):
    def __init__(self, rayss:"List[Rays]"):
        """Container for a list of :class:`Rays`, and have mechanism to
draw them out with a slight path correction. Expected to contain
output of a ray tracing session."""
        self._og = rayss; self.rayss = [copy.copy(rays) for rays in rayss]
    def _draw(self, config):
        oldRays = None
        for rays in self.rayss: rays._draw(config) if oldRays is None else rays._draw(config, np.abs(rays.data[:,5] - oldRays.data[:,5]) < 1e-9); oldRays = rays
        return p5.svg()
    def bounds(self): return self.rayss | cli.op().bounds().all() | cli.T() | cli.toMin() + cli.toMin() + cli.toMax() + cli.toMax() | cli.deref()
    def __copy__(self): return RaysPath([rays for rays in self._og])
    def __len__(self): return len(self.rayss)
    def __getitem__(self, idx): return self.rayss[idx]
    def __repr__(self): return f"<RaysPath #rays={len(self.rayss)}>"
class OpticElement(Drawable):
    surfaces:"List[Surface]"
    def __init__(self):
        """Represents a solid object with several Surfaces"""
        pass
    def __len__(self): return len(self.surfaces)
    def __getitem__(self, idx): return self.surfaces[idx]
    def cast(self, rays, *args, **kwargs): return OpticSystem().add(self).cast(rays, *args, **kwargs)
def signed_area(points):
    n = len(points); area = 0.0
    for i in range(n): j = (i + 1) % n; area += (points[j][0] + points[i][0]) * (points[j][1] - points[i][1])
    return area / 2.0
class Polygon(OpticElement):
    def __init__(self, points=None, **kw):
        """Creates an OpticElement that has the shape of any polygon you want.
Example::

    p = Polygon([[1, 2], [3, 4], [5, 6]])

This will create a prism with verticies at the specified points"""
        if points and signed_area(points) > 0: points = points | cli.reverse() | cli.deref()
        self.points = points; self.surfaces = ([*points, points[0]] | cli.window(2) | ~cli.apply(lambda p1, p2: LineSurface(p1[0], p1[1], p2[0], p2[1], **kw, opticElement=self)) | cli.deref()) if points is not None else None
    def __copy__(self): ans = Polygon(); ans.points = self.points; ans.surfaces = [copy.copy(s) for s in self.surfaces]; return ans
    def bounds(self): return self.surfaces | cli.op().bounds().all() | cli.T() | cli.toMin() + cli.toMin() + cli.toMax() + cli.toMax() | cli.deref()
    def _draw(self, config): [s._draw(config) for s in self.surfaces]; return p5.svg()
    def __repr__(self): return f"<Polygon(OpticElement) #points={len(self.points)} position=({self.points | cli.T() | cli.toMean().all() | cli.aS(round, 2).all() | cli.join(', ')})"
class Lens(OpticElement):
    def __init__(self, x=50, y=0, R1=300, R2=300, thickness=3, height=20, angle=0, surface="arc", **kw):
        """Represents a Lens with center at (x, y), and with 2 radiuses, R1 for left and R2 for right.

:param R1: radius of the left side, works for both arc and parabola mode
:param angle: angle offset of the Lens
:param surface: either "arc" or "parabola", which will make a lens of that geometry"""
        self.x = x; self.y = y; self.R1 = R1; self.R2 = R2; self.thickness = thickness; self.height = height
        dx1 = (-thickness/2 + R1*(1-math.cos(math.asin(height/2/R1)))) if R1 > 0 else -thickness/2
        dx2 = ( thickness/2 - R2*(1-math.cos(math.asin(height/2/R2)))) if R1 > 0 else thickness/2
        c = math.cos(angle); s = math.sin(angle); cx = x; cy = y; trans = lambda x,y: [cx+x*c-y*s, cy+x*s+y*c]
        kw = {"gp1": None, "gp2": None, **kw}; fkw = dict(kw); fkw["gp1"] = kw["gp2"]; fkw["gp2"] = kw["gp1"]
        if surface == "arc": meth = ArcSurface.from2Points
        elif surface == "parabola": meth = PolySurface.parabolaFrom2Points
        else: raise Exception(f"Does not recognize the surface '{surface}'. Only 'arc' and 'parabola' is valid right now")
        self.surfaces = [
            LineSurface(*trans(dx1, height/2), *trans(dx2, height/2), **kw),
            meth(*trans(dx1, -height/2), *trans(dx1, height/2), R1, **(kw if R1 > 0 else fkw)),
            LineSurface(*trans(dx2, -height/2), *trans(dx1, -height/2), **kw),
            meth(*trans(dx2, height/2), *trans(dx2, -height/2), R2, **(kw if R2 > 0 else fkw))
        ]
    def __copy__(self): ans = Lens(); ans.surfaces = [copy.copy(s) for s in self.surfaces]; return ans
    def bounds(self): return self.surfaces | cli.op().bounds().all() | cli.T() | cli.toMin() + cli.toMin() + cli.toMax() + cli.toMax() | cli.deref()
    def _draw(self, config): [s._draw(config) for s in self.surfaces]; return p5.svg()
    def __repr__(self): return f"<Lens R1={self.R1} R2={self.R2} thickness={self.thickness} height={self.height}>"
class OpticSystem(Drawable):
    def __init__(self, elements:"List[OpticElement | Surface]"=None, envGlassParam=gps["air"]):
        self.envGlassParam = envGlassParam; self.opticElements = []; self.rawSurfaces = []; self._debug = []; self.add(*elements or [])
    def add(self, *elements):
        for element in elements:
            if isinstance(element, Surface): self.rawSurfaces.append(copy.copy(element)) # element is copied so that this OpticSystem can freely modify all Surfaces params without worrying that it will affect other setups
            elif isinstance(element, OpticElement): self.opticElements.append(copy.copy(element))
            else: raise Exception(f"Can only add either Surface or OpticElement. Object passed in is of type {type(element)}")
        return self
    def _details(self) -> str:
        count = 0; s = ""
        for e in self.opticElements:
            s += f"{e}\n"
            for sur in e.surfaces: s += f"{count}) {sur}\n"; count += 1
            s += "\n"
        if self.rawSurfaces:
            s += "Raw surfaces:\n"
            for sur in self.rawSurfaces: s += f"{count}) {sur}\n"; count += 1
            s += "\n"
        return s
    @property
    def surfaces(self):
        """Grabs all surfaces from all optic elements and raw surfaces"""
        return [*self.opticElements | cli.op().surfaces.all(), self.rawSurfaces] | cli.joinSt() | cli.deref()
    def cast(self, rays:Rays, passes=20, verbose=False) -> RaysPath:
        rays = copy.copy(rays); surfaces = self.surfaces
        # if verbose: print(f"OpticElements ({len(self.opticElements)} total):"); print(self.opticElements | cli.insId() | ~cli.apply(lambda i,e: f"{i}) {e}") | cli.join("\n"))
        for i, s in enumerate(surfaces):
            a = s.gp1 is None; b = s.gp2 is None
            if a or b:
                if a and not b: s.gp1 = gps["air"]
                elif b and not a: s.gp2 = gps["air"]
                else: raise Exception(f"Surface #{i} {s} have to have at least one specified glass params")
        if verbose: print(self._details())
        # if verbose: print(f"\nSurfaces ({len(surfaces)} total):"); print(surfaces | cli.insId() | ~cli.apply(lambda i,s: f"{i}) {s}") | cli.join("\n"), end="\n\n")
        r = rays; path = [r]; data = []; self._debug = []
        for i in range(passes):
            raysBefore = []; raysAfter = []; data.append([]); self._debug.append([])
            for s in surfaces: rp = copy.copy(r); raysBefore.append(rp); raysAfter.append(s.castS(rp)) # casting to all available surfaces
            raysBefore = np.stack([re.data for re in raysBefore]); raysAfter = np.stack([re.data for re in raysAfter]); self._debug[-1].append([np.copy(raysBefore), np.copy(raysAfter)])
            intersected = raysAfter[:,:,5] - raysBefore[:,:,5]; intersected = (0.9 < intersected) * (intersected < 1.1); self._debug[-1].append(intersected)
            raysBefore[~intersected,3] = float("inf") # If there're no intersections, then set the raysBefore's lengths to inf. Checks for intersection by noting the #transforms
            argmin = raysBefore[:,:,3] | cli.T() | cli.toArgmin().all(); data[-1].append(argmin); self._debug[-1].append(argmin) # then checks for closest interaction
            r.data[:] = raysBefore[argmin, range(raysBefore.shape[1])]; newR = Rays(raysAfter[argmin, range(raysAfter.shape[1])], ogSurface=surfaces[argmin | cli.count() | ~cli.sort() | cli.cut(1) | cli.item()])
            if np.allclose(r.data[:,5], newR.data[:,5]): break # trim away at the paths that no longer change in #transforms
            r = newR; path.append(r)
        if verbose: print(data | cli.cut(0) | cli.pretty() | cli.insId() | ~cli.apply(lambda x,y: "".join([f"Pass: {x}; ", f"chosen surfaces: {y}"])) | cli.join("\n"))
        return RaysPath(path)
    def __getitem__(self, idx): return self.surfaces[idx]
    def __len__(self): return len(self.surfaces)
    def _draw(self, config): config["surfaceIdx2Id"] = {s.idx:f"{i}" for i,s in enumerate(self.surfaces)}; [e._draw(config) for e in self.opticElements]; [s._draw(config) for s in self.surfaces]; return p5.svg()
    def __repr__(self): return f"<OpticSystem #opticElements={len(self.opticElements)} #surfaces={len(self.surfaces)}>"
    def _repr_extra(self): return html.escape(self._details()).replace("\n", "<br>")
    def bounds(self): return [self.opticElements, self.surfaces] | cli.op().bounds().all(2) | cli.joinSt() | cli.T() | cli.toMin() + cli.toMin() + cli.toMax() + cli.toMax() | cli.deref()

In [11]:
l1 = Lens(R1=-20, R2=20, thickness=5, gp2=gps["BK7"], capture=True);
r = Rays.pointToBounds(58, 50, l1.bounds(), color="rainbow", N=10, coverage=0.3)
opsys = OpticSystem().add(l1); rp = opsys.cast(r, verbose=True); Drawables([opsys, rp])

<Lens R1=300 R2=300 thickness=3 height=20>
0) <Line(Surface) (47.5, 10.0, 52.5, 10.0) gp1='air' gp2='BK7' capture=True>
1) <Arc(Surface) x=30.18 y=-0.0 r=20 startAngle(deg)=330.0 endAngle(deg)=390.0 gp1='BK7' gp2='air' capture=True>
2) <Line(Surface) (52.5, -10.0, 47.5, -10.0) gp1='air' gp2='BK7' capture=True>
3) <Arc(Surface) x=35.18 y=-0.0 r=20 startAngle(deg)=330.0 endAngle(deg)=390.0 gp1='air' gp2='BK7' capture=True>


Pass: 0; chosen surfaces: 0   0   0   0   0   0   0   3   3   3   
Pass: 1; chosen surfaces: 1   1   1   2   2   2   2   1   1   1   
Pass: 2; chosen surfaces: 3   3   2   0   0   0   0   3   3   3   
Pass: 3; chosen surfaces: 1   2   0   0   0   0   0   0   0   0   
Pass: 4; chosen surfaces: 2   1   0   0   0   0   0   0   0   0   
Pass: 5; chosen surfaces: 0   0   0   0   0   0   0   0   0   0   


<Drawables #n=2>

In [12]:
%%time
r = Rays.parallel(N=10, y=-4, height=10, theta=0.04, color="rainbow"); ogR = copy.copy(r)
l1 = Lens(x=50, R1=-20, R2=300, thickness=5, gp2=gps["BK7"], capture=True)
l2 = LineSurface(80, -20, 80, 20, gp2=gps["air"], capture=True)
l3 = Lens(x=100, R1=20, R2=300, thickness=5, gp2=gps["BK7"])
l4 = Polygon([[120, -20], [130, 20], [140, -20]], gp2=gps["BK7"], capture=True, mode="mirror")
opsys = OpticSystem().add(l1, l2, l3, l4); rp = opsys.cast(r, verbose=True); Drawables([opsys, rp], {"Surface_showIndex": True})

<Lens R1=300 R2=300 thickness=3 height=20>
0) <Line(Surface) (47.5, 10.0, 52.5, 10.0) gp1='air' gp2='BK7' capture=True>
1) <Arc(Surface) x=30.18 y=-0.0 r=20 startAngle(deg)=330.0 endAngle(deg)=390.0 gp1='BK7' gp2='air' capture=True>
2) <Line(Surface) (52.5, -10.0, 47.5, -10.0) gp1='air' gp2='BK7' capture=True>
3) <Arc(Surface) x=-247.33 y=-0.0 r=300 startAngle(deg)=358.09 endAngle(deg)=361.91 gp1='air' gp2='BK7' capture=True>

<Lens R1=300 R2=300 thickness=3 height=20>
4) <Line(Surface) (100.18, 10.0, 102.33, 10.0) gp1='air' gp2='BK7'>
5) <Arc(Surface) x=117.5 y=0.0 r=20 startAngle(deg)=150.0 endAngle(deg)=210.0 gp1='air' gp2='BK7'>
6) <Line(Surface) (102.33, -10.0, 100.18, -10.0) gp1='air' gp2='BK7'>
7) <Arc(Surface) x=-197.5 y=-0.0 r=300 startAngle(deg)=358.09 endAngle(deg)=361.91 gp1='air' gp2='BK7'>

<Polygon(OpticElement) #points=3 position=(130.0, -6.67)
8) <Line(Surface) (120, -20, 130, 20) gp1='air' gp2='BK7' capture=True opticElement=<Polygon(OpticElement) #points=3 position=(

<Drawables #n=2>

In [13]:
l1 = Lens(R1=-30, R2=20, thickness=5, gp2=gps["BK7"])
r = Rays.parallelToBounds(0, -10, l1.bounds(), color="rainbow", coverage=0.7)
opsys = OpticSystem().add(l1); rp = opsys.cast(r); Drawables([opsys, rp])

<Drawables #n=2>

In [14]:
l1 = Lens(R1=-20, R2=20, thickness=5, gp2=gps["BK7"])
r = Rays.pointToBounds(0, 50, l1.bounds(), color="rainbow", N=10, coverage=0.6)
opsys = OpticSystem().add(l1); rp = opsys.cast(r, verbose=True); Drawables([opsys, rp])

<Lens R1=300 R2=300 thickness=3 height=20>
0) <Line(Surface) (47.5, 10.0, 52.5, 10.0) gp1='air' gp2='BK7'>
1) <Arc(Surface) x=30.18 y=-0.0 r=20 startAngle(deg)=330.0 endAngle(deg)=390.0 gp1='BK7' gp2='air'>
2) <Line(Surface) (52.5, -10.0, 47.5, -10.0) gp1='air' gp2='BK7'>
3) <Arc(Surface) x=35.18 y=-0.0 r=20 startAngle(deg)=330.0 endAngle(deg)=390.0 gp1='air' gp2='BK7'>


Pass: 0; chosen surfaces: 1   1   1   1   1   1   1   1   1   0   
Pass: 1; chosen surfaces: 3   3   3   3   3   3   3   3   3   3   
Pass: 2; chosen surfaces: 0   0   0   0   0   0   0   0   0   2   
Pass: 3; chosen surfaces: 0   0   0   0   0   0   0   0   0   0   


<Drawables #n=2>

In [15]:
r = Rays.parallel(x=0, y=52, theta=-pi/8, color="rainbow")
l1 = LineSurface(30,-40,30,40, gp1=gps["air"], gp2=gps["BK7"]); r1 = l1.castS(r)
l2 = LineSurface(50,-40,50,40, gp1=gps["BK7"], gp2=gps["air"]); r2 = l2.castS(r1)
Drawables([l1, l2, RaysPath([r, r1, r2])])

<Drawables #n=3>

In [16]:
r = Rays.parallel(x=0, y=30, theta=-pi/8); r.data[:,4] = np.linspace(450, 650, 10) # horizontal surface
l1 = LineSurface(10,20,40,20, gp1=gps["air"], gp2=gps["BK7"]); r1 = l1.castS(r)
assert np.allclose(r1.data[:,0], np.array([37.20776529, 34.30429203, 31.40081878, 28.49734552, 25.59387227, 22.69039901, 19.78692576, 16.8834525 , 13.97997925, 11.07650599]))
assert np.allclose(r1.data[:,1], np.array([19.99999999, 19.99999999, 19.99999999, 19.99999999, 19.99999999, 19.99999999, 19.99999999, 19.99999999, 19.99999999, 19.99999999]))
assert np.allclose(r1.data[:,2], np.array([5.36302894, 5.36397438, 5.36479558, 5.36551498, 5.36615018, 5.36671514, 5.3672211 , 5.36767716, 5.36809075, 5.36846802]))
assert np.allclose(r1.data[:,5], np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))
# Drawables([l1, RaysPath([r, r1])])

In [17]:
r = Rays.parallel(x=0, y=30, theta=-pi/8); r.data[:,4] = np.linspace(450, 650, 10) # baseline standard case, vertical
l1 = LineSurface(20,22,20,40, gp1=gps["air"], gp2=gps["BK7"]); r1 = l1.castS(r)
assert np.allclose(r1.data[:,0], np.array([20.00000002, 20.00000002, 20.00000002, 20.00000002, 20.00000002, -0.21260191, -0.63780572, -1.06300953, -1.48821335, -1.91341716]))
assert np.allclose(r1.data[:,1], np.array([27.12768975, 25.92503175, 24.72237375, 23.51971575, 22.31705775, 29.48673359, 28.46020078, 27.43366797, 26.40713515, 25.38060234]))
assert np.allclose(r1.data[:,2], np.array([ 6.02958845,  6.02926665,  6.02898731,  6.02874272,  6.02852686, -0.39269908, -0.39269908, -0.39269908, -0.39269908, -0.39269908]))
assert np.allclose(r1.data[:,5], np.array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.]))
#Drawables([l1, RaysPath([r, r1])])
r = Rays.parallel(x=0, y=30, theta=-pi/8); r.data[:,4] = np.linspace(450, 650, 10) # switched direction of LineSurface
l1 = LineSurface(20,40,20,22, gp1=gps["BK7"], gp2=gps["air"]); r1 = l1.castS(r)
assert np.allclose(r1.data[:,0], np.array([20.00000002, 20.00000002, 20.00000002, 20.00000002, 20.00000002, -0.21260191, -0.63780572, -1.06300953, -1.48821335, -1.91341716]))
assert np.allclose(r1.data[:,1], np.array([27.12768975, 25.92503175, 24.72237375, 23.51971575, 22.31705775, 29.48673359, 28.46020078, 27.43366797, 26.40713515, 25.38060234]))
assert np.allclose(r1.data[:,2], np.array([-0.25359686, -0.25391866, -0.254198  , -0.25444259, -0.25465845, -0.39269908, -0.39269908, -0.39269908, -0.39269908, -0.39269908]))
assert np.allclose(r1.data[:,5], np.array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.]))
# Drawables([l1, RaysPath([r, r1])])

In [18]:
r = Rays.parallel(x=0, y=30, theta=-pi/8); r.data[:,4] = np.linspace(450, 650, 10) # varies x by a little bit from the above tests
l1 = LineSurface(19,22,20,40, gp1=gps["air"], gp2=gps["BK7"]); r1 = l1.castS(r)
assert np.allclose(r1.data[:,0], np.array([19.30095793, 19.23564654, 19.17033514, 19.10502374, 19.03971235, -0.21260191, -0.63780572, -1.06300953, -1.48821335, -1.91341716]))
assert np.allclose(r1.data[:,1], np.array([27.41724246, 26.24163733, 25.06603219, 23.89042706, 22.71482192, 29.48673359, 28.46020078, 27.43366797, 26.40713515, 25.38060234]))
assert np.allclose(r1.data[:,2], np.array([ 6.00904582,  6.00876995,  6.00853047,  6.00832079,  6.00813575, -0.39269908, -0.39269908, -0.39269908, -0.39269908, -0.39269908]))
assert np.allclose(r1.data[:,5], np.array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.]))
#Drawables([l1, RaysPath([r, r1])])
r = Rays.parallel(x=0, y=30, theta=-pi/8); r.data[:,4] = np.linspace(450, 650, 10)
l1 = LineSurface(19,40,20,22, gp1=gps["BK7"], gp2=gps["air"]); r1 = l1.castS(r)
assert np.allclose(r1.data[:,0], np.array([19.70841853, 19.7768066 , 19.84519467, 19.91358274, 19.98197081, -0.21260191, -0.63780572, -1.06300953, -1.48821335, -1.91341716]))
assert np.allclose(r1.data[:,1], np.array([27.24846675, 26.01748149, 24.78649622, 23.55551095, 22.32452569, 29.48673359, 28.46020078, 27.43366797, 26.40713515, 25.38060234]))
assert np.allclose(r1.data[:,2], np.array([-0.23256814, -0.23293605, -0.23325543, -0.23353507, -0.23378188, -0.39269908, -0.39269908, -0.39269908, -0.39269908, -0.39269908]))
assert np.allclose(r1.data[:,5], np.array([1., 1., 1., 1., 1., 0., 0., 0., 0., 0.]))
#Drawables([l1, RaysPath([r, r1])])

In [19]:
r = Rays.parallel(x=40, y=30, theta=pi+pi/8); r.data[:,4] = np.linspace(450, 650, 10) # rays going from right to left
l1 = LineSurface(19,22,20,40, gp1=gps["air"], gp2=gps["BK7"]); r1 = l1.castS(r)
assert np.allclose(r1.data[:,0], np.array([41.91341716, 41.48821335, 41.06300953, 40.63780572, 40.21260191, 39.78739809, 19.06286338, 19.13125145, 19.19963952, 19.26802759]))
assert np.allclose(r1.data[:,1], np.array([25.38060234, 26.40713515, 27.43366797, 28.46020078, 29.48673359, 30.51326641, 23.13154105, 24.36252632, 25.59351158, 26.82449685]))
assert np.allclose(r1.data[:,2], np.array([3.53429174, 3.53429174, 3.53429174, 3.53429174, 3.53429174, 3.53429174, 3.8034179 , 3.8029003 , 3.80243179, 3.80200515]))
assert np.allclose(r1.data[:,5], np.array([0., 0., 0., 0., 0., 0., 1., 1., 1., 1.]))
# Drawables([l1, RaysPath([r, r1])])
r = Rays.parallel(x=40, y=30, theta=pi-pi/8); r.data[:,4] = np.linspace(450, 650, 10) # rays going from right to left, LineSurface switched direction
l1 = LineSurface(20,40,19,22,gp1=gps["air"], gp2=gps["BK7"]); r1 = l1.castS(r)
assert np.allclose(r1.data[:,0], np.array([19.61292454, 19.67823594, 19.74354734, 19.80885873, 19.87417013, 19.93948153, 40.63780572, 41.06300953, 41.48821335, 41.91341716]))
assert np.allclose(r1.data[:,1], np.array([33.03264215, 34.20824728, 35.38385242, 36.55945755, 37.73506268, 38.91066782, 31.53979922, 32.56633203, 33.59286485, 34.61939766]))
assert np.allclose(r1.data[:,2], np.array([9.15063847, 9.1503626 , 9.15012312, 9.14991345, 9.1497284 , 9.14956388, 2.74889357, 2.74889357, 2.74889357, 2.74889357]))
assert np.allclose(r1.data[:,5], np.array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0.]))
# Drawables([l1, RaysPath([r, r1])])

In [20]:
r = Rays.parallel(y=30, x=0, theta=-pi/2, N=10); p = PolySurface(gp2=gps["BK7"], angle=0.2) # for parabola
rp = p.cast(r); r1 = rp[-1]
assert np.allclose(r1.data[:,0], np.array([ 5.00000043,  3.88888925,  2.77777805,  1.66666686,  0.55555566, -0.55555553, -1.66666671, -2.77777788, -3.88888904, -5.0000002 ]))
assert np.allclose(r1.data[:,1], np.array([ 4.42066848,  2.72073646,  1.49307845,  0.65520869,  0.1461775 , -0.08056421, -0.06152106,  0.1740513 ,  0.60227798,  1.20337353]))
assert np.allclose(r1.data[:,2], np.array([5.15638519, 5.07670425, 4.99195913, 4.90532891, 4.82088504, 4.74239671, 4.67228599, 4.61136797, 4.55926052, 4.51496655]))
assert np.allclose(r1.data[:,5], np.array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.]))
# Drawables([rp, p])

In [21]:
%%time
r = Rays.parallel(y=-4, height=13, theta=0.04, color=500); ogR = copy.copy(r); r.data[:,4] = np.linspace(400, 650, len(r.data));
l1 = Lens(x=50, R1=20, R2=300, thickness=5, gp2=gps["BK7"], capture=True)
l2 = LineSurface(80, -20, 80, 20, gp2=gps["air"], capture=True)
l3 = Lens(x=100, R1=20, R2=300, thickness=5, gp2=gps["BK7"])
l4 = Polygon([[120, -20], [130, 20], [140, -20]], gp2=gps["BK7"], capture=True, mode="mirror")
opsys = OpticSystem().add(l1, l2, l3, l4); rp = opsys.cast(r, verbose=True); Drawables([l1, l2, l3, l4, rp])

<Lens R1=300 R2=300 thickness=3 height=20>
0) <Line(Surface) (50.18, 10.0, 52.33, 10.0) gp1='air' gp2='BK7' capture=True>
1) <Arc(Surface) x=67.5 y=0.0 r=20 startAngle(deg)=150.0 endAngle(deg)=210.0 gp1='air' gp2='BK7' capture=True>
2) <Line(Surface) (52.33, -10.0, 50.18, -10.0) gp1='air' gp2='BK7' capture=True>
3) <Arc(Surface) x=-247.5 y=-0.0 r=300 startAngle(deg)=358.09 endAngle(deg)=361.91 gp1='air' gp2='BK7' capture=True>

<Lens R1=300 R2=300 thickness=3 height=20>
4) <Line(Surface) (100.18, 10.0, 102.33, 10.0) gp1='air' gp2='BK7'>
5) <Arc(Surface) x=117.5 y=0.0 r=20 startAngle(deg)=150.0 endAngle(deg)=210.0 gp1='air' gp2='BK7'>
6) <Line(Surface) (102.33, -10.0, 100.18, -10.0) gp1='air' gp2='BK7'>
7) <Arc(Surface) x=-197.5 y=-0.0 r=300 startAngle(deg)=358.09 endAngle(deg)=361.91 gp1='air' gp2='BK7'>

<Polygon(OpticElement) #points=3 position=(130.0, -6.67)
8) <Line(Surface) (120, -20, 130, 20) gp1='air' gp2='BK7' capture=True opticElement=<Polygon(OpticElement) #points=3 position=

<Drawables #n=5>

In [22]:
%%time
r = Rays.parallel(y=-4, height=13, theta=0.04, color=500); ogR = copy.copy(r); r.data[:,4] = np.linspace(400, 650, len(r.data));
l1 = Lens(x=50, R1=20, R2=300, thickness=5, gp2=gps["BK7"], capture=True, surface="parabola")
l2 = LineSurface(80, -20, 80, 20, gp2=gps["air"], capture=True)
l3 = Lens(x=100, R1=20, R2=300, thickness=5, gp2=gps["BK7"])
l4 = Polygon([[120, -20], [130, 20], [140, -20]], gp2=gps["BK7"], capture=True, mode="mirror")
opsys = OpticSystem().add(l1, l2, l3, l4); rp = opsys.cast(r); Drawables([l1, l2, l3, l4, rp])

CPU times: user 73.8 ms, sys: 621 µs, total: 74.4 ms
Wall time: 73 ms


<Drawables #n=5>

In [23]:
l2.captures[2][1].data[:,1] | cli.toMin() & cli.toMax() | cli.deref()

[-0.4538173554203986, 1.8696025982676332]

In [24]:
l2.captures[2][1].hitbox()

In [25]:
l1 = Lens(R1=20, R2=30, thickness=5, gp2=gps["BK7"])
r = Rays.pointToBounds(0, 10, l1.bounds(), color=650, coverage=0.8) + Rays.pointToBounds(0, 0, l1.bounds(), color=550, coverage=0.8) + Rays.pointToBounds(0, -10, l1.bounds(), color=450, coverage=0.8)
opsys = OpticSystem().add(l1); rp = opsys.cast(r)
Drawables([opsys, rp])

<Drawables #n=2>

In [26]:
data = []
for x in range(70, 110):
    l1 = Lens(R1=20, R2=30, thickness=5, gp2=gps["BK7"]); # x = 90
    l2 = LineSurface(x, -10, x, 10, gp2=gps["air"], capture=True)
    N = 30; r = Rays.pointToBounds(0, 0, l1.bounds(), color=650, N=N)# + Rays.pointToBounds(0, 0, l1.bounds(), color=550, N=N) + Rays.pointToBounds(0, 0, l1.bounds(), color=450, N=N)
    opsys = OpticSystem().add(l1, l2); rp = opsys.cast(r, verbose=False)
    data.append([x, l2.captures[2][1].hitbox()])
    # Drawables([opsys, rp])

In [27]:
data | cli.apply(k1lib.fmt.h, 0, level=3) | k1lib.viz.Carousel(2)

In [28]:
l1 = Lens(R1=20, R2=30, thickness=5, gp2=gps["BK7"])
l2 = Lens(R1=-20, R2=30, gp2=gps["BK7"], x=120, y=-3, angle=3*pi/180, mode="mirror")
l3 = Lens(R1=-20, R2=30, gp2=gps["BK7"], x=80, y=-14, angle=210*pi/180, mode="mirror")
r = Rays.pointToBounds(0, 0, l1.bounds(), color=650, coverage=0.35)# + Rays.pointToBounds(0, 0, l1.bounds(), color=550) + Rays.pointToBounds(0, -10, l1.bounds(), color=450)
opsys = OpticSystem().add(l1, l2, l3); rp = opsys.cast(r); r1 = rp.rayss[-1]
assert np.allclose(r1.data[:,0], np.array([82.95582987, 82.24195986, 81.68271961, 81.22577889, 80.82965299, 80.46012634, 80.08809545, 79.68866142, 79.24153584, 78.7331289 ]))
assert np.allclose(r1.data[:,1], np.array([-19.48802751, -18.81643041, -18.24123793, -17.73496775, -17.26642231, -16.80157929, -16.30321416, -15.72955681, -15.03204531, -14.15211141]))
assert np.allclose(r1.data[:,2], np.array([1.16068505, 1.09604303, 1.0438299 , 1.00001574, 0.96097484, 0.92329908, 0.8836409 , 0.83856743, 0.78440983, 0.7170911 ]))
assert np.allclose(r1.data[:,5], np.array([4., 4., 4., 4., 4., 4., 4., 4., 4., 4.]))
# Drawables([opsys, rp])

In [29]:
l1 = Lens(R1=20, R2=30, thickness=5, gp2=gps["BK7"])
l2 = Lens(R1=-20, R2=30, gp2=gps["BK7"], x=120, y=-3, angle=3*pi/180, mode="mirror")
l3 = Lens(R1=20, R2=30, thickness=5, gp2=gps["BK7"], x=80, y=-14, angle=200*pi/180)
r = Rays.pointToBounds(0, 0, l1.bounds(), color=650, coverage=0.35)# + Rays.pointToBounds(0, 0, l1.bounds(), color=550) + Rays.pointToBounds(0, -10, l1.bounds(), color=450)
opsys = OpticSystem().add(l1, l2, l3); rp = opsys.cast(r); r1 = rp.rayss[-1]
assert np.allclose(r1.data[:,0], np.array([80.30388446, 79.80498837, 79.42905221, 79.12952938, 78.87443169, 78.63964009, 78.4057634 , 78.15671284, 77.87934153, 77.5639734 ]))
assert np.allclose(r1.data[:,1], np.array([-20.33699316, -19.49738578, -18.82048263, -18.2497788 , -17.73882467, -17.24578028, -16.73025156, -16.15082761, -15.46252145, -14.61336958]))
assert np.allclose(r1.data[:,2], np.array([3.38541371, 3.40009552, 3.40831993, 3.41303964, 3.41583951, 3.4176741 , 3.4191855 , 3.42086573, 3.42318063, 3.42674789]))
assert np.allclose(r1.data[:,5], np.array([5., 5., 5., 5., 5., 5., 5., 5., 5., 5.]))
# Drawables([opsys, rp])

In [30]:
l1 = Lens(R1=20, R2=30, thickness=5, gp2=gps["BK7"], capture=True)
l3 = Lens(R1=20, R2=30, thickness=5, gp2=gps["BK7"], x=80, y=-14, angle=200*pi/180)
r = Rays.pointToBounds(0, 0, l1.bounds(), color=650, coverage=0.7)# + Rays.pointToBounds(0, 0, l1.bounds(), color=550) + Rays.pointToBounds(0, -10, l1.bounds(), color=450)
opsys = OpticSystem().add(l1, l3); rp = opsys.cast(r); r1 = rp.rayss[-1]
assert np.allclose(r1.data[:,0], np.array([51.66314933, 52.01221184, 52.25714591, 52.41387055, 52.4904979 , 52.4904979 , 52.41387055, 52.25714591, 52.01221184, 51.66314933]))
assert np.allclose(r1.data[:,1], np.array([-7.03638558, -5.38788952, -3.80949715, -2.27164037, -0.75500795, 0.75500795,  2.27164037,  3.80949715,  5.38788952,  7.03638558]))
assert np.allclose(r1.data[:,2], np.array([ 6.46779678,  6.41441929,  6.37117702,  6.33388309,  6.29975274, -0.01656743, -0.05069779, -0.08799171, -0.13123398, -0.18461148]))
assert np.allclose(r1.data[:,5], np.array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2.]))
# Drawables([opsys, rp])

In [31]:
l1 = Lens(R1=20, R2=30, thickness=5, gp2=gps["BK7"])
l2 = Lens(R1=-20, R2=30, gp2=gps["BK7"], x=120, y=-3, angle=3*pi/180, mode="mirror")
l3 = LineSurface(30, -20, 100, -20, gp2=gps["air"], mode="mirror")
r = Rays.pointToBounds(0, 0, l1.bounds(), color="rainbow", coverage=0.35, N=30)
opsys = OpticSystem().add(l1, l2, l3); rp = opsys.cast(r)
Drawables([opsys, rp])

<Drawables #n=2>

In [32]:
l1 = Lens(R1=20, R2=30, thickness=5, gp2=gps["BK7"])
l2 = Lens(R1=-20, R2=30, gp2=gps["BK7"], x=120, y=-3, angle=3*pi/180, mode="mirror")
l3 = LineSurface(30, -20, 100, -20, gp2=gps["air"], mode="mirror")
r = Rays.pointToBounds(0, 0, l1.bounds(), color="rainbow", coverage=0.7, N=30)
opsys = OpticSystem().add(l1, l2, l3); rp = opsys.cast(r)
Drawables([opsys, rp])

<Drawables #n=2>

In [262]:
# import pprint; [0, 1, 2, 5] | cli.apply(lambda i: r1.data[:,i] | cli.aS(pprint.pformat) | cli.op().split("\n") | cli.op().strip().all() | cli.join(" ") | cli.aS(lambda x: f"assert np.allclose(r1.data[:,{i}], np.{x})")) | cli.stdout()

assert np.allclose(r1.data[:,0], np.array([19.61292454, 19.67823594, 19.74354734, 19.80885873, 19.87417013, 19.93948153, 40.63780572, 41.06300953, 41.48821335, 41.91341716]))
assert np.allclose(r1.data[:,1], np.array([33.03264215, 34.20824728, 35.38385242, 36.55945755, 37.73506268, 38.91066782, 31.53979922, 32.56633203, 33.59286485, 34.61939766]))
assert np.allclose(r1.data[:,2], np.array([9.15063847, 9.1503626 , 9.15012312, 9.14991345, 9.1497284 , 9.14956388, 2.74889357, 2.74889357, 2.74889357, 2.74889357]))
assert np.allclose(r1.data[:,5], np.array([1., 1., 1., 1., 1., 1., 0., 0., 0., 0.]))


In [33]:
!../export.py kop --upload=True

2024-02-19 23:10:15,175	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2024-02-19 23:10:15,183	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
14928   0   61%   
9512    1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.5.2
Uninstalling k1lib-1.5.2:
  Successfully uninstalled k1lib-1.5.2
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdi

In [49]:
!../export.py kop

2024-02-21 15:28:02,328	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2024-02-21 15:28:02,335	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- exportAll
14951   0   61%   
9534    1   39%   
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.5.2
Uninstalling k1lib-1.5.2:
  Successfully uninstalled k1lib-1.5.2
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
running bdi

In [5]:
!../export.py kop --bootstrap=True

2024-02-16 20:01:53,995	INFO worker.py:1458 -- Connecting to existing Ray cluster at address: 192.168.1.19:6379...
2024-02-16 20:01:54,002	INFO worker.py:1633 -- Connected to Ray cluster. View the dashboard at 127.0.0.1:8265 
./export started up - /home/kelvin/anaconda3/envs/ray2/bin/python3
----- bootstrapping
Current dir: /home/kelvin/repos/labs/k1lib, /home/kelvin/repos/labs/k1lib/k1lib/../export.py
rm: cannot remove '__pycache__': No such file or directory
Found existing installation: k1lib 1.5.2
Uninstalling k1lib-1.5.2:
  Successfully uninstalled k1lib-1.5.2
running install
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/home/kelvin/anaconda3/envs/ray2/lib/python3.9/site-packages/setuptools/command/easy_install.py:144: EasyInstallDeprecationWarning: easy_install command is deprecated. Use build and pip an